In [85]:
import pandas as pd

In [86]:
df = pd.read_csv("../../data/raw/MBTA_Service_Alerts.csv")
df.head()

,alert_id,gui_mode_name,alert_time_type,effect_name,effect_code,cause_name,cause_code,affent_list,severity_name,severity_code,...,notif_end,created_dt,last_modified_dt,closed_dt,alert_lifecycle,color,service_effect_text,short_header,timeframe_text,ObjectId
0,288839,Access,One-time,Escalator Closure,OTHER_EFFECT,maintenance,MAINTENANCE,"escalator, 396",up to 10 minutes,3,...,2019/01/01 08:05:24+00,2019/01/01 02:05:23+00,2019/01/01 08:05:24+00,2019-01-01T08:05:24Z,New,999999,Aquarium escalator unavailable,Aquarium Escalator 396 (State Street lobby to ...,NaN,1
1,288939,Bus,One-time,Delay,OTHER_EFFECT,medical emergency,MEDICAL_EMERGENCY,NaN,up to 20 minutes,5,...,2019/01/02 16:39:13+00,2019/01/02 15:43:56+00,2019/01/02 16:39:13+00,2019-01-02T16:39:13Z,New,FFC72C,Route 83 delay,Route 83 experiencing delays of up to 20 minut...,NaN,2
2,288840,Bus,One-time,Delay,OTHER_EFFECT,accident,ACCIDENT,NaN,up to 25 minutes,6,...,2019/01/01 07:22:42+00,2019/01/01 06:26:15+00,2019/01/01 07:22:42+00,2019-01-01T07:22:42Z,New,FFC72C,Route 116 delay,Route 116 experiencing delays of up to 25 minu...,NaN,3
3,288940,Bus,One-time,Delay,OTHER_EFFECT,traffic,OTHER_CAUSE,NaN,up to 20 minutes,5,...,2019/01/02 17:57:15+00,2019/01/02 16:14:33+00,2019/01/02 17:57:15+00,2019-01-02T17:57:15Z,New,FFC72C,Route 44 delay,Route 44 experiencing delays of up to 20 minut...,NaN,4
4,288841,Bus,One-time,Delay,OTHER_EFFECT,NaN,UNKNOWN_CAUSE,NaN,up to 20 minutes,5,...,2019/01/01 09:02:44+00,2019/01/01 06:45:15+00,2019/01/01 09:02:44+00,2019-01-01T09:02:44Z,New,7C878E,Silver Line - SL5 delay,Silver Line - SL5 experiencing delays of up to...,NaN,5


In [87]:
# set index col to alert_id
df.set_index('alert_id', inplace=True)

In [88]:
# convert dt cols to datetime objects
datetime_cols = ["notif_start", "notif_end", "created_dt", "last_modified_dt", "closed_dt"]
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)

# convert dt cols to Boston time
df[datetime_cols] = df[datetime_cols].apply(lambda x: x.dt.tz_convert("America/New_York"))

In [89]:
# drop unnecessary cols
df = df.drop(columns=['url', 'timeframe_text', 'header', 'alert_lifecycle', 'details', 'affent_list', 'severity_name'], axis=1)

# drop placeholder rows (rows with colors of gray or white)
# df['color'].value_counts()
df = df[~df['color'].isin(['999999', 'FFFFFF'])]

# rename cols
df = df.rename(columns={'gui_mode_name': 'transit_mode', 
                        'short_header': 'header'})

In [90]:
df

,transit_mode,alert_time_type,effect_name,effect_code,cause_name,cause_code,severity_code,notif_start,notif_end,created_dt,last_modified_dt,closed_dt,color,service_effect_text,header,ObjectId
alert_id,,,,,,,,,,,,,,,,
288939,Bus,One-time,Delay,OTHER_EFFECT,medical emergency,MEDICAL_EMERGENCY,5,2019-01-02 10:43:53-05:00,2019-01-02 11:39:13-05:00,2019-01-02 10:43:56-05:00,2019-01-02 11:39:13-05:00,2019-01-02 11:39:13-05:00,FFC72C,Route 83 delay,Route 83 experiencing delays of up to 20 minut...,2
288840,Bus,One-time,Delay,OTHER_EFFECT,accident,ACCIDENT,6,2019-01-01 01:26:09-05:00,2019-01-01 02:22:42-05:00,2019-01-01 01:26:15-05:00,2019-01-01 02:22:42-05:00,2019-01-01 02:22:42-05:00,FFC72C,Route 116 delay,Route 116 experiencing delays of up to 25 minu...,3
288940,Bus,One-time,Delay,OTHER_EFFECT,traffic,OTHER_CAUSE,5,2019-01-02 11:14:31-05:00,2019-01-02 12:57:15-05:00,2019-01-02 11:14:33-05:00,2019-01-02 12:57:15-05:00,2019-01-02 12:57:15-05:00,FFC72C,Route 44 delay,Route 44 experiencing delays of up to 20 minut...,4
288841,Bus,One-time,Delay,OTHER_EFFECT,NaN,UNKNOWN_CAUSE,5,2019-01-01 01:45:12-05:00,2019-01-01 04:02:44-05:00,2019-01-01 01:45:15-05:00,2019-01-01 04:02:44-05:00,2019-01-01 04:02:44-05:00,7C878E,Silver Line - SL5 delay,Silver Line - SL5 experiencing delays of up to...,5
288941,Commuter Rail,One-time,Delay,OTHER_EFFECT,NaN,UNKNOWN_CAUSE,3,2019-01-02 11:22:20-05:00,2019-01-02 12:21:54-05:00,2019-01-02 11:22:21-05:00,2019-01-02 12:21:54-05:00,2019-01-02 12:21:54-05:00,80276C,Middleborough/Lakeville Line delay,Middleboro Train 017 (3:43pm outbound) is oper...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513235,Bus,One-time,Detour,DETOUR,NaN,UNKNOWN_CAUSE,7,2023-07-28 10:48:13-04:00,2023-07-28 22:30:00-04:00,2023-07-28 10:48:15-04:00,2023-07-28 10:48:15-04:00,NaT,FFC72C,Route 350 detour,Route 350 detoured due to a street closure at ...,222805
513236,Bus,One-time,Delay,OTHER_EFFECT,traffic,OTHER_CAUSE,5,2023-07-28 10:48:34-04:00,2023-07-28 22:30:00-04:00,2023-07-28 10:48:34-04:00,2023-07-28 10:48:34-04:00,NaT,FFC72C,Route 504 delay,Route 504 experiencing delays of up to 20 minu...,222806
513237,Bus,One-time,Delay,OTHER_EFFECT,traffic,OTHER_CAUSE,5,2023-07-28 10:48:45-04:00,2023-07-28 22:30:00-04:00,2023-07-28 10:48:46-04:00,2023-07-28 10:48:46-04:00,NaT,FFC72C,Route 553 delay,Route 553 experiencing delays of up to 20 minu...,222807
